In [78]:
from typing import Iterable
from functools import partial
from itertools import chain, repeat
from datetime import datetime
from multiprocessing.pool import Pool, ThreadPool

from pathlib import Path

import subprocess
import shutil
import pydicom

from typing import Tuple, Iterator, Dict, Optional

import json

In [32]:
sequence = "WIP PDT1_3D 08mm"
sequence_out = "T1"

In [46]:
inputdir = "GRIP"
outputdir = "GRIP_SORTED"

inputdir = Path(inputdir)
outputdir = Path(outputdir)
sequences = {
    "WIP PDT1_3D 08mm": "T13D",
    "WIP DelRec - WIP 2beatpause1mm 3000 HR 21": "Look-Locker",
    "WIP 07mmTE565 3D TSE": "T2"
}

In [37]:
def create_protocol_filemap(inputdir: Path, outputdir: Path, protocol: str, protocol_out: str, n_jobs=None) -> Dict[Path, Path]:
    protocol_filemap = {}
    pool = Pool(n_jobs)
    task = partial(create_study_filemap, output_dir=outputdir, protocol=protocol, protocol_out=protocol_out)
    results = pool.map(task, study_iterator(inputdir))
    return dict(chain(*(x.items() for x in results)))

In [39]:
filemap = create_protocol_filemap(inputdir, outputdir, sequence, sequence_out)

EnteringEnteringEnteringEnteringEntering     GRIP/PAT_001/2023_01_25/S__113730/DICOM/DICOMGRIP/PAT_001/2023_01_26/S__114833/DICOM/DICOMGRIP/PAT_001/2023_01_23/S__112909/DICOM/DICOMGRIP/PAT_001/2023_01_23/S__113593/DICOM/DICOMGRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM






In [40]:
filemap

{PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0179'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0179'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0375'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0375'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0333'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0333'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0419'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0419'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0469'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0469'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0517'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0517'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0403'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0403'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0527'): PosixPath('GRIP_sorte

In [42]:
for src, dest in filemap.items():
    dest.parent.mkdir(exist_ok=True, parents=True)
    shutil.copy(src, dest)

In [18]:
inputdir

PosixPath('GRIP')

In [19]:
patient = inputdir / "PAT_001"

In [21]:
for y in (x for patient in inputdir.iterdir() for date in patient.iterdir() for x in date.iterdir()):
        print(y)

GRIP/PAT_001/2023_01_24/S__113632
GRIP/PAT_001/2023_01_25/S__113730
GRIP/PAT_001/2023_01_26/S__114833
GRIP/PAT_001/2023_01_23/S__112909
GRIP/PAT_001/2023_01_23/S__113593
GRIP/PAT_002/2023_01_23/S__112909


In [89]:
study_dir = inputdir / "PAT_001/2023_01_23/S__112909"

In [90]:
def find_timestamp(x: Path) -> str:
    """TODO: Correct this function."""
    study_data_path = x / "DICOM" / "DICOM"
    first_imfile, _ = next(study_imfiles(study_data_path))
    with pydicom.dcmread(first_imfile) as f:
        timestamp = f.StudyTime
    return timestamp

In [91]:
find_timestamp(study_dir)

'075013'

In [36]:
def is_patient(x: Path) -> bool:
    return x.is_dir() and x.stem[:3] == 'PAT'

def is_imfile(path: Path) -> bool:
    """Checks if path is a dicom IM-file."""
    return path.is_file() and path.stem[:3] == "IM_"

def is_image_subdirectory(path: Path) -> bool:
    """Checks if path is a directory named as a directory,
    meant to find folders which might contain IM_XXXX-files."""
    return path.is_dir() and path.stem.isdigit()

def is_protocol(file: Path, protocol: str) -> bool:
    with pydicom.dcmread(file) as f:
        return protocol == f.ProtocolName

    
def find_timestamp(x: Path) -> str:
    """TODO: Correct this function."""
    return x.stem.split('__')[1]

def study_level_imfiles(study_data_path) -> Iterator[Tuple[Path, int]]:
    """Returns a list of string-representations of the filenames at the top-level of
    of a specific study."""
    return zip(filter(is_imfile, study_data_path.iterdir()), repeat(0))

def subdirectory_imfiles(subdir: Path) -> Iterator[Tuple[Path, int]]:
    return zip(filter(is_imfile, subdir.iterdir()), repeat(int(subdir.stem)))


def renumber_imfile(imfile: Path, offset: int) -> Path:
    """Relabel a file from a subdirectory 000000000N/IM_XXXX -> IM_{2048 * N + XXXX}"""
    return f"IM_{int(imfile.stem.split('_')[1]) + offset * 2048:04d}"

def study_imfiles(study_path: Path):
    return chain(
        study_level_imfiles(study_path),
        *(subdirectory_imfiles(subdir) for subdir in filter(is_image_subdirectory, study_path.iterdir()))        
    )

def filemap_study(study_input: Path, study_output: Path, protocol: str, protocol_out: Optional[str] = None) -> Dict[Path, Path]:
    """Given the path to a specific study, returns a dictionary mapping paths in the original structure
    to the designated new path. Note that the files are NOT copied by this function."""
    filemap = {}
    for imfile, offset in study_imfiles(study_input):
        if is_protocol(imfile, protocol):
            filemap[imfile] = study_output / protocol_out / renumber_imfile(imfile, offset)
    return filemap

In [30]:
# def create_protocol_filemap(inputdir: Path, outputdir: Path, protocol: str, protocol_out: str, n_jobs=None) -> Dict[Path, Path]:
#     protocol_filemap = {}
#     (x for date in patient.iterdir() for x in date.iterdir()))
#     study_iterato/
#     for patient in filter(is_patient, inputdir.iterdir()):
#         protocol_filemap = {
#             **protocol_filemap,
#             **create_patient_filemap(patient, output_dir, protocol, protocol_out)
#         }
#     return protocol_filemap

def create_patient_filemap(patient_dir: Path, output_dir: Path, protocol: str, protocol_out: str) -> Dict[Path, Path]:
    patient_filemap = {}
    for study in (x for date in patient.iterdir() for x in date.iterdir()):
        patient_filemap = {
            **patient_filemap,
            **create_study_filemap(study, output_dir, protocol, protocol_out)
        }
    return patient_filemap


def create_date_filemap(date_dir, output_dir, protocol, protocol_out):
    date_filemap = {}
    patient = date_dir.parent.stem
    for study_dir in date_dir.iterdir():
        create_study_filemap(study_dir, output_dir, protocol, protocol_out)
    return date_filemap


def create_study_filemap(study_dir, output_dir, protocol, protocol_out):
    date_dir = study_dir.parent
    patient = date_dir.parent.stem
    study_data = study_dir / "DICOM" / "DICOM"
    date = datetime.strptime(date_dir.stem, "%Y_%m_%d").strftime("%Y%m%d")
    timestamp = find_timestamp(study_dir)
    study_target = outputdir / patient / f"{date}_{timestamp}"
    print(f"Entering {study_data}"\n")
    return filemap_study(study_data, study_target, protocol, protocol_out)

for y in (x for date in patient.iterdir() for x in date.iterdir()):
    print(y)

def is_imfile(path: Path) -> bool:
    """Checks if path is a dicom IM-file."""
    return path.is_file() and path.stem[:3] == "IM_"

def is_image_subdirectory(path: Path) -> bool:
    """Checks if path is a directory named as a directory,
    meant to find folders which might contain IM_XXXX-files."""
    return path.is_dir() and path.stem.isdigit()

def is_protocol(file: Path, protocol: str) -> bool:
    with pydicom.dcmread(file) as f:
        return protocol == f.ProtocolName

def study_level_imfiles(study_data_path) -> Iterator[Tuple[Path, int]]:
    """Returns a list of string-representations of the filenames at the top-level of
    of a specific study."""
    return zip(filter(is_imfile, study_data_path.iterdir()), repeat(0))

def subdirectory_imfiles(subdir: Path) -> Iterator[Tuple[Path, int]]:
    return zip(filter(is_imfile, subdir.iterdir()), repeat(int(subdir.stem)))


def renumber_imfile(imfile: Path, offset: int) -> Path:
    """Relabel a file from a subdirectory 000000000N/IM_XXXX -> IM_{2048 * N + XXXX}"""
    return f"IM_{int(imfile.stem.split('_')[1]) + offset * 2048:04d}"

def study_imfiles(study_path: Path):
    return chain(
        study_level_imfiles(study_path),
        *(subdirectory_imfiles(subdir) for subdir in filter(is_image_subdirectory, study_path.iterdir()))        
    )

GRIP/PAT_001/2023_01_24/S__113632
GRIP/PAT_001/2023_01_25/S__113730
GRIP/PAT_001/2023_01_26/S__114833
GRIP/PAT_001/2023_01_23/S__112909
GRIP/PAT_001/2023_01_23/S__113593


In [110]:
task = partial(create_study_filemap, output_dir=outputdir, protocol=protocol, protocol_out=protocol_out)
task(study)

Entering GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM


{PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0179'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0179'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0375'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0375'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0333'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0333'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0419'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0419'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0469'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0469'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0517'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0517'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0403'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0403'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0527'): PosixPath('GRIP_sorte

In [121]:
pool = Pool()
task = partial(create_study_filemap, output_dir=outputdir, protocol=protocol, protocol_out=protocol_out)
results = pool.map(task, (x for date in patient.iterdir() for x in date.iterdir()))

EnteringEnteringEnteringEnteringEntering     GRIP/PAT_001/2023_01_25/S__113730/DICOM/DICOMGRIP/PAT_001/2023_01_23/S__113593/DICOM/DICOMGRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOMGRIP/PAT_001/2023_01_26/S__114833/DICOM/DICOMGRIP/PAT_001/2023_01_23/S__112909/DICOM/DICOM






In [129]:
results[0]

{PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0179'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0179'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0375'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0375'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0333'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0333'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0419'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0419'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0469'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0469'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0517'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0517'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0403'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0403'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0527'): PosixPath('GRIP_sorte

In [130]:
for result in chain((x.items() for x in results)):
    print(result)

dict_items([(PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0179'), PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0179')), (PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0375'), PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0375')), (PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0333'), PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0333')), (PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0419'), PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0419')), (PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0469'), PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0469')), (PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0517'), PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0517')), (PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0403'), PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0403')), (PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0527'), Po

In [23]:
mychain = chain(*(x.items() for x in results))

NameError: name 'results' is not defined

In [139]:
dict(mychain)

{PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0179'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0179'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0375'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0375'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0333'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0333'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0419'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0419'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0469'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0469'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0517'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0517'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0403'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0403'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0527'): PosixPath('GRIP_sorte

In [132]:
mychain

In [125]:
results[2]

{PosixPath('GRIP/PAT_001/2023_01_26/S__114833/DICOM/DICOM/IM_0179'): PosixPath('GRIP_sorted/PAT_001/20230126_114833/T1/IM_0179'),
 PosixPath('GRIP/PAT_001/2023_01_26/S__114833/DICOM/DICOM/IM_0375'): PosixPath('GRIP_sorted/PAT_001/20230126_114833/T1/IM_0375'),
 PosixPath('GRIP/PAT_001/2023_01_26/S__114833/DICOM/DICOM/IM_0333'): PosixPath('GRIP_sorted/PAT_001/20230126_114833/T1/IM_0333'),
 PosixPath('GRIP/PAT_001/2023_01_26/S__114833/DICOM/DICOM/IM_0419'): PosixPath('GRIP_sorted/PAT_001/20230126_114833/T1/IM_0419'),
 PosixPath('GRIP/PAT_001/2023_01_26/S__114833/DICOM/DICOM/IM_0469'): PosixPath('GRIP_sorted/PAT_001/20230126_114833/T1/IM_0469'),
 PosixPath('GRIP/PAT_001/2023_01_26/S__114833/DICOM/DICOM/IM_0517'): PosixPath('GRIP_sorted/PAT_001/20230126_114833/T1/IM_0517'),
 PosixPath('GRIP/PAT_001/2023_01_26/S__114833/DICOM/DICOM/IM_0403'): PosixPath('GRIP_sorted/PAT_001/20230126_114833/T1/IM_0403'),
 PosixPath('GRIP/PAT_001/2023_01_26/S__114833/DICOM/DICOM/IM_0527'): PosixPath('GRIP_sorte

In [141]:
for y in (x for patient in inputdir.iterdir() for date in patient.iterdir() for x in date.iterdir()):
    print(y)

GRIP/PAT_001/2023_01_24/S__113632
GRIP/PAT_001/2023_01_25/S__113730
GRIP/PAT_001/2023_01_26/S__114833
GRIP/PAT_001/2023_01_23/S__112909
GRIP/PAT_001/2023_01_23/S__113593


In [44]:
filemap = create_protocol_filemap(inputdir, outputdir, "WIP PDT1_3D 08mm", "T1")

Entering GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM
Entering GRIP/PAT_001/2023_01_25/S__113730/DICOM/DICOM
Entering GRIP/PAT_001/2023_01_26/S__114833/DICOM/DICOM
Entering GRIP/PAT_001/2023_01_23/S__112909/DICOM/DICOM
Entering GRIP/PAT_001/2023_01_23/S__113593/DICOM/DICOM


{}

In [52]:
filemap = create_patient_filemap(Path("GRIP/PAT_001"), outputdir, "WIP PDT1_3D 08mm", "T1")

Entering GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM
Entering GRIP/PAT_001/2023_01_25/S__113730/DICOM/DICOM
Entering GRIP/PAT_001/2023_01_26/S__114833/DICOM/DICOM
Entering GRIP/PAT_001/2023_01_23/S__112909/DICOM/DICOM
Entering GRIP/PAT_001/2023_01_23/S__113593/DICOM/DICOM


In [53]:
filemap

{PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0179'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0179'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0375'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0375'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0333'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0333'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0419'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0419'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0469'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0469'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0517'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0517'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0403'): PosixPath('GRIP_sorted/PAT_001/20230124_113632/T1/IM_0403'),
 PosixPath('GRIP/PAT_001/2023_01_24/S__113632/DICOM/DICOM/IM_0527'): PosixPath('GRIP_sorte

In [33]:
sequence = "WIP PDT1_3D 08mm"
study_path = Path("GRIP/PAT_001/2023_01_23/S__112909/")
study_data_path = study_path / "DICOM/DICOM"
target_dir = Path(f"GRIP_SORTED/PAT_001/230123_000000/{sequence}/")

In [41]:
inpath = Path("GRIP/PAT_001/2023_01_23/S__112909/DICOM/DICOM")
outpath = Path("GRIP_SORTED/PAT_001/20230123_112909/")
filemap = filemap_study(inpath, outpath, sequence, "T1")

In [ ]:
for src, dest in filemap:
    shutil.copy(src, dest)

In [ ]:
int(imfile.parent.stem)

In [ ]:
def lower_level_imfiles(study_data_path):
    return sum([subgrouped_imfiles(subdir) for subdir in filter(lambda x: x.is_dir() and x.isdigit(), study_data_path.)])

# Deal with entire DICOMDIR

In [ ]:
inpath = Path("GRIP/PAT_001/2023_01_23/S__112909/DICOM/DICOMDIR")
inpath

In [ ]:
counter = 0
for el in ds:
    print(el)
    for e in el:
        print(e)
    counter += 1
    if counter > 10000:
        break

In [ ]:
ds.ProtocolName

In [ ]:
ds = pydicom.dcmread(inpath)
pydicom.FileDataset(inpath, dsa
                   )

In [ ]:
ds

In [ ]:
fs = pydicom.FileDataset(inpath / "DICOMDIR")

In [ ]:
fs

In [43]:
outputdir

PosixPath('GRIP_sorted')

In [44]:
from sort_mri import *

In [82]:
def sorted_study_iterator(sorted_dir: Path) -> Iterator[Path]:
    return (study for patient in sorted_dir.iterdir() for study in patient.iterdir())

def sorted_study_metadata(study_path):
    study_metadata = {}
    for path in filter(lambda x: x.is_dir(), study_path.iterdir()):
        protocol_filelist = list(map(lambda x: int(x.stem.split("_")[1]), filter(is_imfile, path.iterdir())))
        study_metadata[path.stem] = {
            "min": min(protocol_filelist),
            "max": max(protocol_filelist),
            "num_files": len(protocol_filelist)
        }
    return study_metadata

def store_study_metadata(study_path: Path) -> None:
    filepath = study / "info.json"
    with open(study / "info.json", "w") as f:
        json.dump(sorted_study_metadata(study), f, indent=4)

def add_sorted_metadata(output_dir: Path) -> None:
    for study in sorted_study_iterator(output_dir):
        store_study_metadata(study)

In [83]:
add_sorted_metadata(outputdir)